In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 9.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
def get_nonlinear_equations_with_confounder(n_vars, max_lag):
    """Get nonlinear equations with confounder U"""
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * X1[t-2]^2 - 0.1 * X1[t-2]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * X1[t-4]^2 - 0.1 * X1[t-4]^3 + e4",
                "X3[t] = 0.35 * X4[t]^2 - 0.15 * X4[t]^3 + 0.2 * X2[t-3]^2 + 0.3 * U[t]^2 + e3",
                "X2[t] = 0.3 * X3[t-1]^2 - 0.05 * X3[t-1]^3 + e2",
                "X1[t] = 0.4 * X2[t]^2 - 0.2 * X2[t]^3 + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
    elif n_vars == 6:
        base_equations = get_nonlinear_equations_with_confounder(4, max_lag)
        additional = [
            "X6[t] = 0.45 * X5[t]^2 - 0.15 * X5[t]^3 + 0.4 * U[t]^2 + e6",
            "X5[t] = 0.3 * X4[t-1]^2 - 0.1 * X4[t-1]^3 + e5"
        ]
        return additional + base_equations
    elif n_vars == 8:
        base_equations = get_nonlinear_equations_with_confounder(6, max_lag)
        additional = [
            "X8[t] = 0.4 * X7[t]^2 - 0.12 * X7[t]^3 + 0.35 * U[t]^2 + e8",
            "X7[t] = 0.35 * X6[t-1]^2 - 0.08 * X6[t-1]^3 + e7"
        ]
        return additional + base_equations
    return []

class MixedNonlinearConfoundedGenerator:
    def __init__(self, noise_mix_ratio=0.5, noise_params={'scale': 0.1}, random_state=None):
        """
        Initialize generator with mixed Gaussian and Laplace noise and confounder

        Parameters:
        noise_mix_ratio: float between 0 and 1, proportion of Gaussian noise
        noise_params: dict with 'scale' parameter for noise distributions
        random_state: int, random seed
        """
        self.noise_mix_ratio = noise_mix_ratio
        self.noise_params = noise_params
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)

    def generate_mixed_noise(self, size):
        """Generate mixed Gaussian and Laplace noise"""
        mask = np.random.random(size) < self.noise_mix_ratio
        gaussian_noise = np.random.normal(0, self.noise_params['scale'], size=size)
        laplace_noise = np.random.laplace(0, self.noise_params['scale'], size=size)
        return np.where(mask, gaussian_noise, laplace_noise)

    def generate_irregular_timestamps(self, n_points, total_time, min_gap=0.1):
        """Generate irregular sampling times"""
        times = np.zeros(n_points)
        times[0] = np.random.uniform(0, min_gap)

        for i in range(1, n_points):
            gap = np.random.exponential(scale=(total_time-times[i-1])/(n_points-i))
            times[i] = times[i-1] + max(gap, min_gap)

            if times[i] > total_time:
                times = times * (total_time / times[i])

        return times

    def find_nearest_lag_idx(self, timestamps, current_idx, lag_time):
        """Find index of nearest available past observation for given lag"""
        target_time = timestamps[current_idx] - lag_time
        past_timestamps = timestamps[:current_idx]
        if len(past_timestamps) == 0:
            return 0
        return (np.abs(past_timestamps - target_time)).argmin()

    def generate_equations(self, t, X, U, lag_indices, n_vars, max_lag):
        """Execute nonlinear polynomial equations with confounder"""
        noise = self.generate_mixed_noise(n_vars + 1)  # +1 for U
        equations = get_nonlinear_equations_with_confounder(n_vars, max_lag)
        true_links = extract_coefficients_from_equations(equations)

        # Generate U first as it's a confounder
        U[t] = noise[-1]

        # Generate values based on nonlinear relationships
        for i in range(n_vars-1, -1, -1):
            var_name = f'X{i+1}'
            value = 0

            # Add causal influences with polynomial terms
            for (source, lag, target, power), coef in true_links.items():
                if target == var_name:
                    if source == 'U':
                        value += coef * (U[t] ** power)
                    else:
                        source_idx = int(source[1:]) - 1
                        if lag == 0:
                            value += coef * (X[t, source_idx] ** power)
                        else:
                            lag_idx = lag_indices[abs(lag)-1]
                            value += coef * (X[lag_idx, source_idx] ** power)

            # Add noise term
            X[t, i] = value + noise[i]

    def generate_multivariate_ts(self, n_points, n_vars, max_lag, total_time=100, min_gap=0.1):
        """Generate multivariate time series with irregular sampling and confounder"""
        # Initialize arrays
        X = np.zeros((n_points, n_vars))
        U = np.zeros(n_points)  # Array for confounder U

        # Generate irregular timestamps
        timestamps = self.generate_irregular_timestamps(n_points, total_time, min_gap)

        # Initialize first steps with noise
        for i in range(max_lag):
            X[i] = self.generate_mixed_noise(n_vars)
            U[i] = self.generate_mixed_noise(1)[0]

        # Generate time series
        for t in range(max_lag, n_points):
            mean_diff = np.mean(np.diff(timestamps))
            lag_indices = [self.find_nearest_lag_idx(timestamps, t, i * mean_diff)
                         for i in range(1, max_lag + 1)]

            self.generate_equations(t, X, U, lag_indices, n_vars, max_lag)

        # Create DataFrame
        columns = [f'X{i+1}' for i in range(n_vars)] + ['U']
        df = pd.DataFrame(np.column_stack([X, U[:, np.newaxis]]), columns=columns)
        df['time'] = timestamps

        return df

def extract_coefficients_from_equations(equations):
    """Extract coefficients and relationships from equations including U"""
    causal_links = {}

    for eq in equations:
        if '=' not in eq:
            continue

        left, right = [side.strip() for side in eq.split('=')]
        if 'e' in right and len(right.split('+')) == 1:
            continue  # Skip pure noise equations

        target = left.split('[')[0]
        terms = [term.strip() for term in right.split('+')]

        for term in terms:
            if '*' not in term or not ('X' in term or 'U' in term):
                continue

            parts = term.split('*')
            coeff = float(parts[0].strip())
            var_part = parts[1].strip()
            base_var = var_part.split('^')[0] if '^' in var_part else var_part
            var = base_var.split('[')[0]

            power = 1
            if '^' in var_part:
                power = int(var_part.split('^')[1].split(' ')[0])

            if var == 'U':
                lag = 0
            else:
                lag_part = base_var.split('[')[1].split(']')[0]
                lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])

            causal_links[(var, lag, target, power)] = coeff

    return causal_links

def extract_linear_links_for_graph(equations):
    """Extract linear causal links for graph visualization"""
    links = {}

    for eq in equations:
        if '=' in eq:
            left, right = [side.strip() for side in eq.split('=')]
            target = left.split('[')[0]

            if target != 'U':  # Skip U's equation
                terms = [term.strip() for term in right.split('+')]
                for term in terms:
                    if '*' in term and ('X' in term or 'U' in term):
                        parts = term.split('*')
                        coeff = float(parts[0].strip())
                        var_part = parts[1].strip()
                        var = var_part.split('^')[0].split('[')[0]

                        if var == 'U':
                            lag = 0
                        else:
                            lag_part = var_part.split('[')[1].split(']')[0]
                            lag = 0 if lag_part == 't' else -int(lag_part.split('-')[1])

                        links[(var, lag, target)] = coeff

    return links

def save_dataset_and_graph(df, n_vars, max_lag, sample_size, gaussian_ratio, output_dir="output_mixed_confounded"):
    """Save dataset and create causal graph visualization"""
    gaussian_pct = int(gaussian_ratio * 100)
    laplace_pct = 100 - gaussian_pct
    dir_path = f"{output_dir}/gaussian_{gaussian_pct}_laplace_{laplace_pct}"
    Path(dir_path).mkdir(parents=True, exist_ok=True)

    base_filename = f'{dir_path}/mixed_confounded_ts_n{sample_size}_vars{n_vars}_lag{max_lag}'

    # Save dataset
    df.to_csv(f'{base_filename}.csv', index=False)

    # Get equations and extract links
    equations = get_nonlinear_equations_with_confounder(n_vars, max_lag)
    true_links = extract_linear_links_for_graph(equations)

    # Create variable names including U
    var_names = [f'X{i+1}' for i in range(n_vars)] + ['U']
    n_total_vars = n_vars + 1
    val_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1))
    graph_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1), dtype='bool')

    # Fill matrices based on true links
    for (source, lag, target), weight in true_links.items():
        if source == 'U':
            source_idx = n_vars
        else:
            source_idx = int(source[1:]) - 1

        target_idx = int(target[1:]) - 1
        lag_idx = abs(lag)

        # Add the link to the matrices
        val_matrix[source_idx, target_idx, lag_idx] = weight
        graph_matrix[source_idx, target_idx, lag_idx] = True

        # For contemporaneous links, make val_matrix symmetric
        if lag == 0:
            val_matrix[target_idx, source_idx, lag_idx] = weight

    # Plot and save causal graph
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Nonlinear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Nonlinear Causal Graph with Confounder\n(n={sample_size}, vars={n_vars}, lag={max_lag})\nGaussian: {gaussian_pct}%, Laplace: {laplace_pct}%')
    plt.savefig(f'{base_filename}_graph.png')
    plt.close()

    # Plot time series
    plt.figure(figsize=(15, 10))
    for col in df.columns[:-2]:  # Exclude U and time
        plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f'Time Series (n={sample_size}, vars={n_vars}, lag={max_lag})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_filename}_series.png')
    plt.close()

    # Plot sampling intervals
    plt.figure(figsize=(10, 6))
    time_diffs = np.diff(df['time'])
    plt.hist(time_diffs, bins=50, density=True)
    plt.title('Distribution of Sampling Intervals')
    plt.xlabel('Time Difference')
    plt.ylabel('Density')
    plt.grid(True)
    plt.savefig(f'{base_filename}_sampling.png')
    plt.close()

    # Save causal structure description
    with open(f'{base_filename}_structure.txt', 'w') as f:
        f.write(f"True Nonlinear Causal Structure with Confounder (Gaussian: {gaussian_pct}%, Laplace: {laplace_pct}%):\n")
        f.write("Format: (source, lag, target) => weight\n")
        f.write("\nEquations:\n")
        for eq in equations:
            f.write(f"{eq}\n")
        f.write("\nCausal Links:\n")
        for link, weight in true_links.items():
            f.write(f"{link} => {weight}\n")

def analyze_mixed_confounded_data(df, title="Mixed Error Time Series Analysis with Confounder"):
    """Analyze and plot the time series data with confounder"""
    print(f"\nAnalyzing {title}")
    print("=" * 50)

    # Time Series Plot
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 1, 1)
    for col in df.columns:
        if col != 'time':
            plt.plot(df['time'], df[col], label=col, alpha=0.7)
    plt.title(f"{title} - Time Series")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)

    # Sampling Intervals
    plt.subplot(2, 1, 2)
    time_diffs = np.diff(df['time'])
    plt.hist(time_diffs, bins=50, density=True)
    plt.title("Distribution of Sampling Intervals")
    plt.xlabel("Time Difference")
    plt.ylabel("Density")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Summary Statistics
    print("\nSummary Statistics:")
    print("-" * 30)
    print(df.drop('time', axis=1).describe())

    # Correlation Analysis
    print("\nCorrelation with Confounder U:")
    print("-" * 30)
    u_corr = df.drop('time', axis=1).corr()['U'].sort_values(ascending=False)
    print(u_corr)

def generate_all_combinations():
    """Generate datasets for all combinations"""
    sample_sizes = [500, 1000, 3000, 5000]
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]
    mix_ratios = [0.3, 0.5, 0.7]

    for n in sample_sizes:
        for vars in n_vars_list:
            for lag in max_lags:
                for ratio in mix_ratios:
                    gaussian_pct = int(ratio * 100)
                    laplace_pct = 100 - gaussian_pct
                    print(f"\nGenerating dataset: n={n}, vars={vars}, lag={lag}, gaussian={gaussian_pct}%, laplace={laplace_pct}%")

                    generator = MixedNonlinearConfoundedGenerator(
                        noise_mix_ratio=ratio,
                        noise_params={'scale': 0.1},
                        random_state=42
                    )

                    df = generator.generate_multivariate_ts(
                        n_points=n,
                        n_vars=vars,
                        max_lag=lag,
                        total_time=100,
                        min_gap=0.1
                    )

                    # Save dataset and create visualizations
                    save_dataset_and_graph(
                        df=df,
                        n_vars=vars,
                        max_lag=lag,
                        sample_size=n,
                        gaussian_ratio=ratio
                    )
                    print("Dataset and visualizations saved successfully")

if __name__ == "__main__":
    print("Generating mixed error time series with confounder...")

    # # Example case
    # n_points = 1000
    # n_vars = 4
    # max_lag = 2
    # ratio = 0.7

    # generator = MixedNonlinearConfoundedGenerator(
    #     noise_mix_ratio=ratio,
    #     noise_params={'scale': 0.1},
    #     random_state=42
    # )

    # df = generator.generate_multivariate_ts(
    #     n_points=n_points,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     total_time=100,
    #     min_gap=0.1
    # )

    # # Save and analyze example case
    # save_dataset_and_graph(
    #     df=df,
    #     n_vars=n_vars,
    #     max_lag=max_lag,
    #     sample_size=n_points,
    #     gaussian_ratio=ratio
    # )

    # analyze_mixed_confounded_data(df, "Mixed Gaussian-Laplace Example with Confounder")

    # Generate all combinations
    generate_all_combinations()

Generating mixed error time series with confounder...

Generating dataset: n=500, vars=4, lag=2, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, gaussian=70%, laplace=30%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, gaussian=70%, laplace=30%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=500

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-3-3f8de2056720>:269: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))
<ipython-input-3-3f8de2056720>:281: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly close

Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=2, gaussian=70%, laplace=30%


<ipython-input-3-3f8de2056720>:256: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=3, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=3, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=3, gaussian=70%, laplace=30%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=4, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=4, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=8, lag=4, gaussian=70%, laplace=30%
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=2, gaussian=30%, laplace=70%
Dataset and visualizations saved successfully

Generating dataset: n=1000, vars=4, lag=2, gaussian=50%, laplace=50%
Dataset and visualizations saved successfully

Generating dataset: n=1000, var

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [5]:
!zip -r /content/output_B2C.zip /content/output_mixed_confounded

  adding: content/output_mixed_confounded/ (stored 0%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/ (stored 0%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n5000_vars4_lag2_structure.txt (deflated 46%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n3000_vars8_lag4_structure.txt (deflated 59%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n500_vars6_lag3_graph.png (deflated 4%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n3000_vars4_lag4_sampling.png (deflated 25%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n3000_vars4_lag3_series.png (deflated 3%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n500_vars4_lag2_series.png (deflated 2%)
  adding: content/output_mixed_confounded/gaussian_30_laplace_70/mixed_confounded_ts_n3000_vars6_

In [6]:
from google.colab import files
files.download('/content/output_B2C.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>